In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import math
import numpy
import numpy as np
import random
from PIL import Image
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('max_steps', 2000, 'Number of steps to run trainer.')
flags.DEFINE_integer('hidden1', 128, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_integer('batch_size', 4, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data '
                     'for unit testing.')


In [3]:
NUM_CLASSES = 2 
IMAGE_SIZE = 28 
CHANNELS = 3
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE * CHANNELS


In [4]:
# Hidden Layer1
def inference(images, hidden1_units, hidden2_units):
  
    with tf.name_scope('hidden1'):
        weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units],
                                                  stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
                              name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),name='biases')
        hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
        
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(tf.truncated_normal([hidden2_units, NUM_CLASSES],
                                                  stddev=1.0 / math.sqrt(float(hidden2_units))),
                              name='weights')
        biases = tf.Variable(tf.zeros([NUM_CLASSES]),name='biases')
        logits = tf.matmul(hidden2, weights) + biases
    return logits

    

In [5]:

def cal_loss(logits, labels):
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss


In [6]:
def training(loss, learning_rate):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op

In [7]:
def evaluation(logits, labels):
    correct = tf.nn.in_top_k(logits, labels, 1)
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [8]:
def placeholder_inputs(batch_size):
    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
    return images_placeholder, labels_placeholder

In [9]:
def fill_feed_dict(images_feed,labels_feed, images_pl, labels_pl):
    feed_dict = {images_pl: images_feed,labels_pl: labels_feed,}
    return feed_dict

In [10]:
def do_eval(sess,eval_correct,images_placeholder,labels_placeholder,data_set):
  # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = 4 // FLAGS.batch_size
    num_examples = steps_per_epoch * FLAGS.batch_size
    for step in xrange(steps_per_epoch):
        feed_dict = fill_feed_dict(train_images,train_labels,images_placeholder,labels_placeholder)
        true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = true_count / num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %(num_examples, true_count, precision))

In [11]:
# Get the sets of images and labels for training, validation, and
train_images = []
for filename in ['/home/affine/DeepLearningProject/Beach/ImageNet5.jpg','/home/affine/DeepLearningProject/Beach/ImageNet2.jpg','/home/affine/DeepLearningProject/Beach/ImageNet4.jpg', '/home/affine/DeepLearningProject/Beach/ImageNet3.jpg']:
    image = Image.open(filename)
    image = image.resize((IMAGE_SIZE,IMAGE_SIZE))
    print(image)
    train_images.append(np.array(image))
    
train_images = np.array(train_images)
train_images = train_images.reshape(4,IMAGE_PIXELS)

label = [0,1,1,1]
train_labels = np.array(label)


<PIL.Image.Image image mode=RGB size=28x28 at 0x7FCBDE9FB780>
<PIL.Image.Image image mode=RGB size=28x28 at 0x7FCBDE9FB780>
<PIL.Image.Image image mode=RGB size=28x28 at 0x7FCBDE9FB780>
<PIL.Image.Image image mode=RGB size=28x28 at 0x7FCBDE9FB780>


In [12]:
def run_training():
  # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
    # Generate placeholders for the images and labels.
        images_placeholder, labels_placeholder = placeholder_inputs(4)

    # Build a Graph that computes predictions from the inference model.
        logits = inference(images_placeholder,FLAGS.hidden1,FLAGS.hidden2)
    # Add to the Graph the Ops for loss calculation.
        loss = cal_loss(logits, labels_placeholder)

    # Add to the Graph the Ops that calculate and apply gradients.
        train_op = training(loss, FLAGS.learning_rate)

    # Add the Op to compare the logits to the labels during evaluation.
        eval_correct = evaluation(logits, labels_placeholder)

    # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
        sess = tf.Session()

    # Run the Op to initialize the variables.
        init = tf.initialize_all_variables()
        sess.run(init)
    for step in xrange(FLAGS.max_steps):
        start_time = time.time()
        feed_dict = fill_feed_dict(train_images,train_labels,images_placeholder,labels_placeholder)
        _, loss_value = sess.run([train_op, loss],feed_dict=feed_dict)
        duration = time.time() - start_time
        if step % 100 == 0:
        # Print status to stdout.
            print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
        if (step + 1) % 1000 == 0 or (step + 1) == FLAGS.max_steps:
            saver.save(sess, FLAGS.train_dir, global_step=step)
            print('Training Data Eval:')
            do_eval(sess,eval_correct,images_placeholder,labels_placeholder,train_images)


In [13]:
def main(_):
    run_training()
if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Step 0: loss = 0.02 (0.010 sec)
Step 100: loss = 0.62 (0.003 sec)
Step 200: loss = 0.58 (0.004 sec)
Step 300: loss = 0.57 (0.002 sec)
Step 400: loss = 0.57 (0.002 sec)
Step 500: loss = 0.56 (0.003 sec)
Step 600: loss = 0.56 (0.003 sec)
Step 700: loss = 0.56 (0.002 sec)
Step 800: loss = 0.56 (0.002 sec)
Step 900: loss = 0.56 (0.003 sec)
Training Data Eval:
  Num examples: 4  Num correct: 3  Precision @ 1: 0.7500
Step 1000: loss = 0.56 (0.002 sec)
Step 1100: loss = 0.56 (0.003 sec)
Step 1200: loss = 0.56 (0.003 sec)
Step 1300: loss = 0.56 (0.003 sec)
Step 1400: loss = 0.56 (0.003 sec)
Step 1500: loss = 0.56 (0.002 sec)
Step 1600: loss = 0.56 (0.003 sec)
Step 1700: loss = 0.56 (0.003 sec)
Step 1800: loss = 0.56 (0.003 sec)
Step 1900: loss = 0.56 (0.003 sec)
Training Data Eval:
  Num examples: 4  Num correct: 3  Precision @ 1: 0.7500


SystemExit: 

/home/affine/.conda/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
columns = ['a','ab','date1','date2']
for i in columns:
    match = re.match("date(.+)", i)
    print(match)


['a', 'ab']


['a', 'ab']
